# Imports

In [190]:
import numpy as np
import os
import sys
import tensorflow as tf
import cv2

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image


# This is needed since the notebook is stored in the object_detection folder.
sys.path.append(r"C:\Users\Igor\Documents\TensorFlow\models\research")
sys.path.append(r"C:\Users\Igor\Documents\TensorFlow\models\research\object_detection")
sys.path.append(r"C:\Users\Igor\Documents\TensorFlow\models\research\object_detection\ssd_mobilenet_v1_coco\saved_model")
from object_detection.utils import ops as utils_ops

## Env setup

In [191]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [192]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

In [193]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = r'C:\Users\Igor\Documents\TensorFlow\models\research\object_detection\ssd_mobilenet_v1_coco\saved_model\frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = r'C:\Users\Igor\Documents\TensorFlow\models\research\object_detection\ssd_mobilenet_v1_coco\saved_model\object-detection.pbtxt'

## Load a (frozen) Tensorflow model into memory.

In [194]:
sys.path.append(r'C:\Users\Igor\Documents\TensorFlow\models\research\object_detection\ssd_mobilenet_v1_coco\saved_model\graph.pbtxt')
PATH_TO_FROZEN_GRAPH = r'C:\Users\Igor\Documents\TensorFlow\models\research\object_detection\ssd_mobilenet_v1_coco\saved_model\frozen_inference_graph.pb'
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [195]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [196]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [197]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = r'C:\Users\Igor\Desktop\numbers'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}.jpg'.format(i)) for i in range(1, 454) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (24, 18)

In [198]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [199]:
def Writing_i_k_Crops(Xmin, Xmax, Ymin, Ymax, k):
    res = []
    heigh = []
    width = []
    H = []
    W = []
    print(k)
    for i in range(len(Xmin)):
        res.append([Xmin[i], Ymin[i], Xmax[i], Ymax[i]])        

        res = sorted(res, key=lambda r:r[3])
    

    for i in range(len(res)):
        width.append(res[i][2] - res[i][0])
        heigh.append(res[i][3] - res[i][1])
        
        w = res[i][2] - res[i][0]
        W.append(w)
        h = res[i][3] - res[i][1]
        H.append(h)


        pts1 = np.float32([[res[i][0], res[i][1]], [res[i][2], res[i][1]], [res[i][2], res[i][3]], [res[i][0], res[i][3]]])
        pts2 = np.float32([[0,0],[W[i],0],[W[i],H[i]],[0,H[i]]])

        M = cv2.getPerspectiveTransform(pts1,pts2)

        img = cv2.imread(image_path,3)

        dst = cv2.warpPerspective(img,M,(W[i], H[i]))
        H_dst, W_dst = np.shape(dst)[:2]

        if W_dst < 300:
            cv2.imwrite(r'C:\Users\Igor\Documents\TensorFlow\models\research\object_detection\test_images\crop\{}.jpg'.format(str(k)+'_'+str(i)), dst)
    

In [200]:
k = 0
for image_path in TEST_IMAGE_PATHS:
  k = k+1
  image = cv2.imread(image_path)


  #Режем image на две части, тк модель принимает изображения 300на300
    
    
  H, W = np.shape(image)[:2]

  image1 = image.copy()[0:int(1*H/2), :, :]
  image2 = image.copy()[int(1*H/2):int(H), :, :]
  

  #Запускаем модель для первой половины image


  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np1 = image1
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded1 = np.expand_dims(image_np1, axis=0)
  


    
    
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np1, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np1,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=3)
    
    
    
    
      # Ниже извлекаем координаты ректанглов  
  Xmax1 =[]
  Xmin1 =[]
  Ymax1 =[]
  Ymin1 =[]    
  for i, box in enumerate(output_dict['detection_boxes']):
    if output_dict['detection_scores'][i] > 0.30:
        

    
        h, w = np.shape(image1)[:2]
        
       
        y1 = int(box[0]*h)
        x1 = int(box[1]*w)
        y2 = int(box[2]*h)
        x2 = int(box[3]*w)
        
        #Добавляем padding
        deltaX = 0.05
        deltaY = 0.15
        
        y1new = round(int(y1 - (y2 - y1)*deltaY))
        x1new = round(int(x1 - (x2 - x1)*deltaX))
        y2new = round(int(y2 + (y2 - y1)*deltaY))
        x2new = round(int(x2 + (x2 - x1)*deltaX))
        
        Xmax1.append(x2new)
        Xmin1.append(x1new)
        Ymax1.append(y2new)
        Ymin1.append(y1new)


    
  #Повторяем для второй половины image  

    
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np2 = image2
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded2 = np.expand_dims(image_np2, axis=0)

    
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np2, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np2,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=3)    
    
  #Извлечение координат производим по двум половинам по-отдельности, кроп по  оригиналу, к игрековым координатам второй половины добавляем половину игрек шейпа
  Xmax2 =[]
  Xmin2 =[]
  Ymax2 =[]
  Ymin2 =[]    
  for i, box in enumerate(output_dict['detection_boxes']):
    if output_dict['detection_scores'][i] > 0.30:
        

    
        h, w = np.shape(image2)[:2]
        
       
        y1 = int(box[0]*h) + int(np.shape(image)[0]/2)
        x1 = int(box[1]*w)
        y2 = int(box[2]*h) + int(np.shape(image)[0]/2)
        x2 = int(box[3]*w)
        
        #Добавляем padding
        deltaX = 0.05
        deltaY = 0.15
        
        y1new = round(int(y1 - (y2 - y1)*deltaY))
        x1new = round(int(x1 - (x2 - x1)*deltaX))
        y2new = round(int(y2 + (y2 - y1)*deltaY))
        x2new = round(int(x2 + (x2 - x1)*deltaX))
        
        Xmax2.append(x2new)
        Xmin2.append(x1new)
        Ymax2.append(y2new)
        Ymin2.append(y1new)

  Xmin = np.concatenate((Xmin1, Xmin2), axis=0)      
  Xmax = np.concatenate((Xmax1, Xmax2), axis=0)
  Ymin = np.concatenate((Ymin1, Ymin2), axis=0)      
  Ymax = np.concatenate((Ymax1, Ymax2), axis=0)
    

  Writing_i_k_Crops(Xmin, Xmax, Ymin, Ymax, k)



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
